# GPU check pytorch

In [1]:
import torch
import subprocess

print("=== PyTorch GPU 확인 ===")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.current_device()}")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    
    # GPU 메모리 정보
    gpu_memory = torch.cuda.get_device_properties(0).total_memory
    print(f"GPU 총 메모리: {gpu_memory / 1024**3:.1f} GB")
    
    # 현재 메모리 사용량
    allocated = torch.cuda.memory_allocated(0)
    cached = torch.cuda.memory_reserved(0)
    print(f"할당된 메모리: {allocated / 1024**2:.1f} MB")
    print(f"캐시된 메모리: {cached / 1024**2:.1f} MB")
    
    # 간단한 GPU 텐서 테스트
    print("\n=== GPU 텐서 테스트 ===")
    x = torch.randn(3, 3).cuda()
    y = torch.randn(3, 3).cuda()
    z = torch.matmul(x, y)
    print(f"GPU 텐서 계산 성공: {z.device}")
    
else:
    print("❌ CUDA를 사용할 수 없습니다.")

print("\n=== nvidia-smi 출력 ===")
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print(result.stdout)
except Exception as e:
    print(f"nvidia-smi 실행 실패: {e}")

=== PyTorch GPU 확인 ===
PyTorch 버전: 2.0.1
CUDA 사용 가능: True
CUDA 버전: 11.7
GPU 개수: 1
현재 GPU: 0
GPU 이름: NVIDIA GeForce RTX 2080 Ti
GPU 총 메모리: 11.0 GB
할당된 메모리: 0.0 MB
캐시된 메모리: 0.0 MB

=== GPU 텐서 테스트 ===
GPU 텐서 계산 성공: cuda:0

=== nvidia-smi 출력 ===
Sun Sep 14 17:35:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  |   00000000:01:00.0  On |              

In [2]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import torch

# GPU 사용 가능 확인
print("=== GPU 환경 확인 ===")
print(f"PyTorch CUDA: {torch.cuda.is_available()}")
print(f"XGBoost 버전: {xgb.__version__}")

# XGBoost GPU 지원 확인 (다른 방법)
try:
    # GPU 히스토그램 방법이 지원되는지 테스트
    temp_model = xgb.XGBRegressor(tree_method='gpu_hist', gpu_id=0)
    print("XGBoost GPU: 지원됨 (gpu_hist 사용 가능)")
except Exception as e:
    print(f"XGBoost GPU: 확인 불가 ({str(e)[:50]}...)")

print(f"LightGBM 버전: {lgb.__version__}")
print(f"CatBoost 버전: {CatBoostRegressor().get_param('iterations')}")  # 버전 확인 대신

# 가상 회귀 데이터 생성 (주가 데이터 시뮬레이션)
# 대용량 데이터로 GPU 성능 확인
print("=== 대용량 데이터 GPU 테스트 ===")

# 더 큰 데이터셋 생성
n_samples_large = 500000  # 50만 개 (10배 증가)
n_features_large = 100    # 100개 피처

print(f"대용량 데이터 생성 중... ({n_samples_large:,} × {n_features_large})")
np.random.seed(42)

X_large = np.random.randn(n_samples_large, n_features_large)
y_large = (100 + 
           np.sum(X_large[:, :20] * np.random.randn(20), axis=1) + 
           np.random.randn(n_samples_large) * 0.1)

X_train, X_test, y_train, y_test = train_test_split(
    X_large, y_large, test_size=0.2, random_state=42
)

print(f"훈련 데이터: {X_train.shape}")
print(f"메모리 크기: {X_train.nbytes / 1024**2:.1f} MB")

=== GPU 환경 확인 ===
PyTorch CUDA: True
XGBoost 버전: 3.0.4
XGBoost GPU: 지원됨 (gpu_hist 사용 가능)
LightGBM 버전: 4.6.0
CatBoost 버전: None
=== 대용량 데이터 GPU 테스트 ===
대용량 데이터 생성 중... (500,000 × 100)
훈련 데이터: (400000, 100)
메모리 크기: 305.2 MB


In [3]:
# 1. XGBoost CPU vs GPU 테스트
print("\n=== XGBoost 테스트 ===")

# CPU 버전
print("CPU 버전 학습 중...")
start_time = time.time()
xgb_cpu = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    tree_method='hist',  # CPU
    random_state=42,
    verbosity=0
)
xgb_cpu.fit(X_train, y_train)
cpu_time = time.time() - start_time
cpu_pred = xgb_cpu.predict(X_test)
cpu_rmse = np.sqrt(mean_squared_error(y_test, cpu_pred))
cpu_r2 = r2_score(y_test, cpu_pred)

print(f"XGBoost CPU - 시간: {cpu_time:.2f}초, RMSE: {cpu_rmse:.4f}, R²: {cpu_r2:.4f}")

# GPU 버전 시도
print("GPU 버전 학습 중...")
try:
    start_time = time.time()
    xgb_gpu = xgb.XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.1,
        tree_method='gpu_hist',  # GPU
        gpu_id=0,
        random_state=42,
        verbosity=0
    )
    xgb_gpu.fit(X_train, y_train)
    gpu_time = time.time() - start_time
    gpu_pred = xgb_gpu.predict(X_test)
    gpu_rmse = np.sqrt(mean_squared_error(y_test, gpu_pred))
    gpu_r2 = r2_score(y_test, gpu_pred)
    
    print(f"XGBoost GPU - 시간: {gpu_time:.2f}초, RMSE: {gpu_rmse:.4f}, R²: {gpu_r2:.4f}")
    print(f"🚀 속도 향상: {cpu_time/gpu_time:.2f}배")
    
except Exception as e:
    print(f"❌ XGBoost GPU 실행 실패: {e}")


=== XGBoost 테스트 ===
CPU 버전 학습 중...
XGBoost CPU - 시간: 16.00초, RMSE: 0.4871, R²: 0.9887
GPU 버전 학습 중...
XGBoost GPU - 시간: 4.57초, RMSE: 0.4869, R²: 0.9887
🚀 속도 향상: 3.50배


In [4]:
# 3. CatBoost 테스트
print("\n=== CatBoost 테스트 ===")

# CPU 버전
print("CPU 버전 학습 중...")
start_time = time.time()
cat_cpu = CatBoostRegressor(
    iterations=300,
    depth=6,
    learning_rate=0.1,
    task_type='CPU',
    random_seed=42,
    verbose=False
)
cat_cpu.fit(X_train, y_train)
cpu_time = time.time() - start_time
cpu_pred = cat_cpu.predict(X_test)
cpu_rmse = np.sqrt(mean_squared_error(y_test, cpu_pred))
cpu_r2 = r2_score(y_test, cpu_pred)

print(f"CatBoost CPU - 시간: {cpu_time:.2f}초, RMSE: {cpu_rmse:.4f}, R²: {cpu_r2:.4f}")

# GPU 버전 시도
print("GPU 버전 학습 중...")
try:
    start_time = time.time()
    cat_gpu = CatBoostRegressor(
        iterations=300,
        depth=6,
        learning_rate=0.1,
        task_type='GPU',
        devices='0',
        random_seed=42,
        verbose=False
    )
    cat_gpu.fit(X_train, y_train)
    gpu_time = time.time() - start_time
    gpu_pred = cat_gpu.predict(X_test)
    gpu_rmse = np.sqrt(mean_squared_error(y_test, gpu_pred))
    gpu_r2 = r2_score(y_test, gpu_pred)
    
    print(f"CatBoost GPU - 시간: {gpu_time:.2f}초, RMSE: {gpu_rmse:.4f}, R²: {gpu_r2:.4f}")
    print(f"🚀 속도 향상: {cpu_time/gpu_time:.2f}배")
    
except Exception as e:
    print(f"❌ CatBoost GPU 실행 실패: {e}")

print("\n=== 모든 테스트 완료! ===")


=== CatBoost 테스트 ===
CPU 버전 학습 중...
CatBoost CPU - 시간: 18.32초, RMSE: 0.2641, R²: 0.9967
GPU 버전 학습 중...
CatBoost GPU - 시간: 11.33초, RMSE: 0.2773, R²: 0.9963
🚀 속도 향상: 1.62배

=== 모든 테스트 완료! ===


# LGBM
### LightGBM은 보통 CPU 전용 빌드
- 그래서 GPU를 쓰려면 직접 소스에서 컴파일해야 합니다.
- LightGBM GPU 버전도 있긴 하지만, 안정성과 속도 측면에서 XGBoost, CatBoost GPU 버전이 더 자주 쓰입니다.
- 특히 CatBoost는 GPU 지원이 훨씬 잘 되어 있어서 대규모 데이터엔 선호도가 높음.

In [5]:
# LightGBM GPU 테스트 (개선된 버전)
print("\n=== LightGBM GPU 테스트 (개선) ===")

# OpenCL 정보 확인
import subprocess
try:
    result = subprocess.run(['clinfo'], capture_output=True, text=True, timeout=10)
    if "Platform" in result.stdout:
        print("✅ OpenCL 플랫폼 발견")
        print(result.stdout[:200] + "...")
    else:
        print("❌ OpenCL 플랫폼 없음")
except:
    print("❌ clinfo 명령어 실행 실패")

# GPU 버전 재시도 (더 많은 옵션)
print("GPU 버전 학습 재시도...")
try:
    # 방법 1: 기본 GPU 설정
    start_time = time.time()
    lgb_gpu = lgb.LGBMRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.1,
        device='gpu',
        gpu_use_dp=False,  # 단정밀도 사용
        random_state=42,
        verbose=-1
    )
    lgb_gpu.fit(X_train, y_train)
    gpu_time = time.time() - start_time
    gpu_pred = lgb_gpu.predict(X_test)
    gpu_rmse = np.sqrt(mean_squared_error(y_test, gpu_pred))
    gpu_r2 = r2_score(y_test, gpu_pred)
    
    print(f"✅ LightGBM GPU - 시간: {gpu_time:.2f}초, RMSE: {gpu_rmse:.4f}, R²: {gpu_r2:.4f}")
    print(f"🚀 속도 향상: {cpu_time/gpu_time:.2f}배")
    
except Exception as e1:
    print(f"❌ 방법 1 실패: {e1}")
    
    # 방법 2: CUDA 백엔드 시도
    try:
        start_time = time.time()
        lgb_gpu2 = lgb.LGBMRegressor(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.1,
            device='cuda',  # CUDA 직접 사용
            random_state=42,
            verbose=-1
        )
        lgb_gpu2.fit(X_train, y_train)
        gpu_time = time.time() - start_time
        gpu_pred = lgb_gpu2.predict(X_test)
        gpu_rmse = np.sqrt(mean_squared_error(y_test, gpu_pred))
        gpu_r2 = r2_score(y_test, gpu_pred)
        
        print(f"✅ LightGBM CUDA - 시간: {gpu_time:.2f}초, RMSE: {gpu_rmse:.4f}, R²: {gpu_r2:.4f}")
        print(f"🚀 속도 향상: {cpu_time/gpu_time:.2f}배")
        
    except Exception as e2:
        print(f"❌ 방법 2도 실패: {e2}")
        print("💡 LightGBM은 CPU 버전으로 사용하고 XGBoost와 CatBoost GPU를 활용하세요")


=== LightGBM GPU 테스트 (개선) ===
❌ OpenCL 플랫폼 없음
GPU 버전 학습 재시도...
❌ 방법 1 실패: No OpenCL device found
❌ 방법 2도 실패: CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
💡 LightGBM은 CPU 버전으로 사용하고 XGBoost와 CatBoost GPU를 활용하세요


[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
